# Use Cohere Command R/R+ to answer questions from data in local (FAISS) vector index - Langchain

You can use Cohere Command R or Cohere Command R+ models deployed in Azure AI and Azure ML with `langchain` to create more sophisticated intelligent applications.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for the Cohere family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Cohere Command R` or `Cohere Command R+` and `Cohere-embed-v3-english` or `Cohere-embed-v3-multilingual`.
* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

### Install Dependencies

The following example demonstrate how to create a Facebook AI Similarity Search (Faiss) that uses a Cohere Embed v3 - English and Cohere Embed v3 - Multilingual models deployed in Azure AI and Azure ML. The chain has been configured with a `CohereChat` and `CohereEmbeddings`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/integrations/chat/cohere/).

In [ ]:
! pip install --quiet langchain langchain_cohere faiss-cpu

In [ ]:
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain_core.documents.base import Document
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage

## Constants
We will set the values for the keys and models that we will use in our RAG pipeline

Let's create instances of our Chat and Embed models deployed in Azure AI or Azure ML. Use the `langchain_cohere` package and configure it as follows:

- `embed_endpoint` and `command_endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.

In [ ]:
# Embed
azure_cohere_embed_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_embed_key = "<key>"

# Command
azure_cohere_command_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_command_key = "<key>"

Prepare the data and format it for vectorstore FAISS

In [ ]:
# Set up the embedding model to be used in the vector index
embed_model = CohereEmbeddings(
    base_url=azure_cohere_embed_endpoint, cohere_api_key=azure_cohere_embed_key
)

In [ ]:
project_descriptions = [
    "Project Alpha: Implementing a new CRM system to improve customer relationships.",
    "Project Beta: Developing an AI-based forecasting tool for sales trends.",
    "Project Gamma: Overhauling the company website for better user engagement.",
    "Project Delta: Integrating blockchain technology for enhanced supply chain transparency.",
    "Project Epsilon: Launching a cloud migration strategy to enhance data accessibility and security.",
    "Project Zeta: Implementing machine learning algorithms to optimize inventory management systems.",
    "Project Eta: Developing a mobile application to improve field service operations and customer reporting.",
    "Project Theta: Upgrading cybersecurity measures to protect against emerging digital threats.",
    "Project Iota: Creating a virtual reality training program to enhance employee skill sets and engagement.",
    "Project Kappa: Automating financial processes using AI to increase efficiency and reduce errors.",
]

project_docs = []
for doc in project_descriptions:
    project_docs.append(Document(page_content=doc, metadata={"source": "local"}))

Ingest documents with Embed endpoint

In [ ]:
# create the vector store using FAISS
db = FAISS.from_documents(project_docs, embed_model)

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 2})

### Create our Chat function
Next, we will create a basic chat function using the `ChatCohere` class

Cohere's `chat endpoint` can accept documents directly and will return a grounded answer that includes citations against those documents.

No prompt is needed since the `Cohere` model will automatically use a RAG prompt when documents are passed in.

Because we are using `LangChain's expression language (LCEL)`, we will also wrap our function at the end with a `RunableLambda` function. Learn more about LCEL here: https://python.langchain.com/docs/expression_language/get_started/

In [ ]:
def ask(inputs):
    """
    Ask a question to the chatbot, expecting a dictionary with 'question' and 'documents'.

    Args:
        inputs (dict): A dictionary containing 'question' and 'documents'.

    Returns:
        str: The response from the chatbot.
    """
    question = inputs["question"]
    documents = inputs["documents"]

    chatbot = ChatCohere(
        base_url=azure_cohere_command_endpoint,
        cohere_api_key=azure_cohere_command_key,
    )

    response = chatbot(messages=[HumanMessage(content=question)], documents=documents)
    return response


ask = RunnableLambda(ask)

## Create the Chain
We can now create the chain by chaining the retriever and the chat function together

In [ ]:
# create the answer chain using our search function and chat function
answer_chain = (
    {"documents": retriever, "question": RunnablePassthrough()}
    | ask
    | StrOutputParser()
)

## Run our chain
You can now run the full chain and see the response as an `AIMessage` object. This will contain the models answer along with citations from the documents retrieved from `FAISS`

In [ ]:
answer_chain.invoke("What AI based projects do we have?")